# Croatia

In [1]:
import pandas as pd
import geopandas as gpd

## Fetch results

In [2]:
df = pd.read_csv('../../data/raw/hrv.csv', encoding='windows-1250',
                 sep=';', skipfooter=79, engine='python')

In [3]:
df = df.rename(columns={'Grad/općina/država': 'muni'})

In [4]:
df.muni = df.muni.replace('MARTIJANEC', 'DONJI MARTIJANEC')
df.muni = df.muni.replace('GRAD ZAGREB', 'ZAGREB')
df.muni = df.muni.replace('OPATIJA', 'OPATIJA/VEPRINAC')
df.muni = df.muni.replace('KONJŠČINA', 'KONJŠĆINA')
df.muni = df.muni.replace('BUDINŠČINA', 'BUDINŠĆINA')
df.muni = df.muni.replace('RAB', 'LOPAR/SUPETARSKA DRAGA')
df.muni = df.muni.replace('VINODOLSKA OPĆINA', 'VINODOLSKA')
df.muni = df.muni.replace('SVETA MARIJA', 'SVETA MARIJA NA MURI')
df.muni = df.muni.replace('POVLJANA', 'POVLJANE')

df.loc[df['Naziv BM'] == 'ILOVIK', 'muni'] = 'ILOVIK'
df.loc[df['Naziv BM'] == 'PREMANTURA', 'muni'] = 'PREMANTURA'

In [5]:
# Get rid of Italian names
df.muni = df.muni.str.split(' - ', expand=True)[0]

In [6]:
data = df.groupby('muni')['Glasovalo birača'].sum() / df.groupby('muni')['Ukupno birača'].sum()

In [7]:
data = data.reset_index().rename(columns={'muni': 'name', 0: 'turnout'})

In [8]:
data.turnout = data.turnout * 100

## Merge

In [9]:
geo = gpd.read_file('../../data/shapefiles/hrv/gadm36_HRV_2.shp')

In [10]:
geo['name'] = geo.NAME_2.str.upper()

In [12]:
geo.shape, data.shape, geo[~geo.name.isin(data.name)].shape, data[~data.name.isin(geo.name)].shape

((560, 15), (554, 2), (11, 15), (9, 2))

In [13]:
geo = geo.merge(data, on='name', how='left')

In [14]:
geo = geo[['name', 'geometry', 'turnout']]

## Export

In [15]:
geo['country'] = 'hrv'

In [60]:
geo.to_file('../../data/processed/hrv')